In [1]:
!pip install dspy pydantic openai

import os
import dspy
from pydantic import BaseModel
from typing import List

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 17.2 MB/s eta 0:00:00


dspy.Signature : Décrit une tâche (inputs/outputs).

dspy.Predict(Signature) : Programme qui fait exécuter la tâche par le LLM.

dspy.ReAct(Signature, tools=[...]) : Crée un agent intelligent qui peut raisonner, choisir des outils, et produire une sortie.

dspy.configure(lm=...) : Définit quel modèle de langage est utilisé.

Outils Python : Fonctions normales que l’agent peut invoquer.

Appel d’agent : agent(user_request="...") → déclenche tout le pipeline.

#A NOTER :

Les fonctions tools ont besoin d'annotations (docstring """, type hints ->) pour faciliter leur interprétation par le LLM.

dspy.Signature définit une tâche ; dspy.Predict(Signature) transforme la tâche en programme exécutable par le LLM.

DSPy génère automatiquement un prompt à partir des inputs et de la dosctring pour chaque tool.



In [11]:
# Création de données factices pour test
class JobOffer(BaseModel):
    title: str
    company: str
    description: str
    site: str

class UserProfile(BaseModel):
    name: str
    skills: List[str]
    experience: str
    interests: List[str]

# Profil factice
user_profile = UserProfile(
    name="Alice Dupont",
    skills=["Python", "Data Science", "Machine Learning"],
    experience="3 ans en analyse de données dans le secteur bancaire",
    interests=["IA appliquée", "analyse prédictive"]
)


# Offres factices
def fetch_jobs_from_site1() -> List[JobOffer]:
    return [
        JobOffer(title="Data Scientist", company="TechCorp", description="Analyse de données clients, Python, ML", site="Site1"),
        JobOffer(title="Développeur Web", company="WebAgency", description="React, CSS, intégration frontend", site="Site1"),
    ]

def fetch_jobs_from_site2() -> List[JobOffer]:
    return [
        JobOffer(title="Machine Learning Engineer", company="AI Solutions", description="Deep learning, mise en production de modèles ML", site="Site2"),
        JobOffer(title="Chef de projet IT", company="ConsultingPro", description="Gestion d’équipe, coordination de projets informatiques", site="Site2"),
    ]



In [12]:
# Création des tools qu'utilisera l'agent
# Important de spécifier ce que fait l'outil pour l'agent

"""
profile: UserProfile = dspy.InputField() signifie :

profile = nom du champ (clé d’entrée).

UserProfile = type de données attendu.

dspy.InputField() = ce champ est une entrée fournie au LLM.
"""

class MatchOfferSignature(dspy.Signature):
    """Déterminer si une offre d'emploi correspond au profil utilisateur."""
    profile: UserProfile = dspy.InputField()
    job: JobOffer = dspy.InputField()
    is_match: bool = dspy.OutputField(desc="True si le poste correspond au profil, False sinon.")

class PitchSignature(dspy.Signature):
    """Rédige un pitch personnalisé (style lettre de motivation)
    adapté au profil et à l'offre d'emploi donnée."""
    profile: UserProfile = dspy.InputField()
    job: JobOffer = dspy.InputField()
    pitch: str = dspy.OutputField()

def semantic_match_offer(profile: UserProfile, job: JobOffer) -> bool:
    result = semantic_matcher(profile=profile, job=job)
    return result.is_match

def generate_pitch(profile: UserProfile, job: JobOffer) -> str:
    """Génère un pitch personnalisé avec DSPy."""
    result = pitch_generator(profile=profile, job=job)
    return result.pitch



In [7]:
# Configuration du LLM
#os.environ["OPENAI_API_KEY"] = ici, spécifiez votre clef API
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini")) # Ou un autre LLM

In [8]:
# Définition de l'agent DSPy
class JobAgent(dspy.Signature):
    user_request: str = dspy.InputField()
    result: str = dspy.OutputField()

In [9]:
# Programme DSPy qui instancie le matching
semantic_matcher = dspy.Predict(MatchOfferSignature)
# Programme DSPy pour générer le pitch
pitch_generator = dspy.Predict(PitchSignature)

# On assemble l’agent avec nos outils
agent = dspy.ReAct(
    JobAgent,
    tools=[fetch_jobs_from_site1, fetch_jobs_from_site2, semantic_match_offer, generate_pitch]
)


In [ ]:
# Utilisation de l'agent
response = agent(user_request="Récupère les offres et propose un pitch pour celles qui correspondent au profil d'Alice")
print(response.result)